In [26]:
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = "retina"

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))


/tmp/ipykernel_270228/945712042.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [1]:
import pandas as pd
import os
import argparse
from typing import Any, Callable, Sequence
from pathlib import Path
from PIL import Image
import matplotlib.image as mpimg

import jax
import jax.random as jr
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax.flatten_util import ravel_pytree
import numpy as np
import matplotlib.pyplot as plt
from flax import linen as nn


from bong.util import run_rebayes_algorithm, gaussian_kl_div, MLP
from bong.src import bbb, blr, bog, bong, experiment_utils
#from bong.agents import AGENT_NAMES




In [21]:

x = [11*n + (n+1)*1 for n in [10,20,30,40,50]]
#x = [11*n + (n+1)*1 for n in [10,40,70,100]]

#x = [11*n + (n+1)*1 for n in [10, 500,1000, 2000, 5000]] # DLR

print(x)

time_ef0 = np.array([29, 181, 547])/500
time_ef1 = np.array([5, 26, 545])/500
print(time_ef0)
print(time_ef1)


[121, 241, 361, 481, 601]
[0.058 0.362 1.094]
[0.01  0.052 1.09 ]


In [19]:
results_dir = '/teamspace/studios/this_studio/jobs/timingfc2'
df = pd.read_csv(f'{results_dir}/summary.csv')


df.head(n=20)

,jobname,agent_name,agent_full_name,model_name,elapsed,summary
0,timingfc2-00,bong-fc-EF0-MC100,bong-fc-R0-Lin0-EF0-MC100-I1-LR0_0,mlp_10_1[P=121],29.376710,"NLL-PI: nan, NLPD-MC: nan"
1,timingfc2-01,bong-fc-EF0-MC100,bong-fc-R0-Lin0-EF0-MC100-I1-LR0_0,mlp_40_1[P=481],191.969056,"NLL-PI: nan, NLPD-MC: nan"
2,timingfc2-02,bong-fc-EF0-MC100,bong-fc-R0-Lin0-EF0-MC100-I1-LR0_0,mlp_70_1[P=841],547.186077,"NLL-PI: nan, NLPD-MC: nan"
3,timingfc2-04,bong-fc-EF1-MC100,bong-fc-R0-Lin0-EF1-MC100-I1-LR0_0,mlp_10_1[P=121],5.284585,"NLL-PI: 1.4670, NLPD-MC: 1.4944"
4,timingfc2-05,bong-fc-EF1-MC100,bong-fc-R0-Lin0-EF1-MC100-I1-LR0_0,mlp_40_1[P=481],26.280944,"NLL-PI: 1.4636, NLPD-MC: 1.5153"
5,timingfc2-06,bong-fc-EF1-MC100,bong-fc-R0-Lin0-EF1-MC100-I1-LR0_0,mlp_70_1[P=841],545.327132,"NLL-PI: nan, NLPD-MC: nan"
6,timingfc2-08,bong-fc-lin,bong-fc-R0-Lin1-EF99-MC99-I1-LR0_0,mlp_10_1[P=121],1.022760,"NLL-PI: 1.4647, NLPD-MC: 2.2098"
7,timingfc2-09,bong-fc-lin,bong-fc-R0-Lin1-EF99-MC99-I1-LR0_0,mlp_40_1[P=481],1.349110,"NLL-PI: 1.4876, NLPD-MC: 5.9302"
8,timingfc2-10,bong-fc-lin,bong-fc-R0-Lin1-EF99-MC99-I1-LR0_0,mlp_70_1[P=841],1.624817,"NLL-PI: 1.4862, NLPD-MC: 11.8617"
9,timingfc2-11,bong-fc-lin,bong-fc-R0-Lin1-EF99-MC99-I1-LR0_0,mlp_100_1[P=1201],1.952296,"NLL-PI: 1.5643, NLPD-MC: 16.3525"


In [6]:
results_dir = '/teamspace/studios/this_studio/jobs/sweep3'
df = pd.read_csv(f'{results_dir}/jobs_with_eval.csv')

#condition = (df['minscore'] != df[args.metric])
condition = (df['lin']==1) | (df['ef']==0)
indices_to_drop = df[condition].index
df_filtered = df.drop(indices_to_drop)

df_filtered.head(n=20)


,jobname,algo,param,lin,dlr_rank,ef,nsample,niter,lr,nlpd_te_final,nlpd_te_mid,nlpd_val_final,nlpd_val_mid
1,sweep3-01,bog,fc,0,99,1,100,99,0.0001,2.539317e+00,3.269134e+00,2.526866e+00,2.753690e+00
3,sweep3-03,bog,fc,0,99,1,100,99,0.0005,1.805015e+00,2.016883e+00,1.855236e+00,1.803196e+00
5,sweep3-05,bog,fc,0,99,1,100,99,0.0010,1.712355e+00,2.092516e+00,1.773460e+00,1.962286e+00
7,sweep3-07,bog,fc,0,99,1,100,99,0.0050,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20
9,sweep3-09,bog,fc,0,99,1,100,99,0.0100,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20
11,sweep3-11,bog,fc,0,99,1,100,99,0.0500,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20
13,sweep3-13,bog,fc,0,99,1,100,99,0.1000,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20


In [18]:

#x = [11*n + (n+1)*1 for n in [10,20,30,40,50]]
x = [11*n + (n+1)*1 for n in [10,40,70,100]]

#x = [11*n + (n+1)*1 for n in [10, 500,1000, 2000, 5000]] # DLR

print(x)

[121, 481, 841, 1201]


In [16]:
# https://gaussianprocess.org/gpml/data/

def make_sarcos_data(ntrain, nval, ntest):
    import scipy.io
    folder = '/teamspace/studios/this_studio/bong/bong/data'
    print(folder)

    mat_data = scipy.io.loadmat(f'{folder}/sarcos_inv.mat') # (44484, 28)
    data_train = mat_data['sarcos_inv']
    max_ntrain = data_train.shape[0] 
    assert ntrain < max_ntrain
    idx_tr = np.arange(0, ntrain)
    X_tr = data_train[idx_tr, :21]
    Y_tr = data_train[idx_tr, 21] # column 22

    idx_val = np.arange(ntrain, ntrain+nval)
    X_val = data_train[idx_val, :21]
    Y_val = data_train[idx_val, 21] # column 22

    mat_data = scipy.io.loadmat(f'{folder}/sarcos_inv_test.mat') # (4449, 28)
    data_test = mat_data['sarcos_inv_test']
    max_ntest = data_test.shape[0] 
    assert ntest < max_ntest
    if ntest == 0: ntest = max_ntest # Use full test set
    idx_te = np.arange(0, ntest)
    X_te = data_test[idx_te, :21]
    Y_te = data_test[idx_te, 21] # column 22

    name = 'sarcos'
    # We return X_tr etc for use by rebayes, as well as X_train (full data) for debugging
    data = {
        'X_tr': X_tr, 'Y_tr': Y_tr, 'X_val': X_val, 'Y_val': Y_val, 'X_te': X_te, 'Y_te': Y_te, 'name': name, 
        'X_train': data_train[:, :21], 'Y_train': data_train[:, 22],
        'X_test': data_test[:, :21], 'Y_test': data_test[:, 22],
        'sarcos': data_train[:, :22]
    }
    return data

In [ ]:
#sarcos = pd.read_csv('sarcos_inv.csv', header = None).values
data = make_sarcos_data(ntrain=1000, nval=1000, ntest=0)

In [36]:

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# https://gaussianprocess.org/gpml/chapters/RW2.pdf
# . The inputs were linearly rescaled to have zero mean and unit variance on the training set.
# The outputs were centered so as to have zero mean on the training set.

scaler = preprocessing.StandardScaler().fit(data['X_train'])
Xtrain, Xtest = scaler.transform(data['X_train']), scaler.transform(data['X_test'])
ytrain, ytest = data['Y_train'], data['Y_test']
mu_y, v_y = np.mean(ytrain), np.var(ytrain)
ytrain, ytest = ytrain - mu_y, ytest - mu_y

def calc_mse(prediction, Y):
    mse = np.mean(np.square(prediction - Y))
    return mse

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(Xtrain, ytrain)
prediction = model.predict(Xtest)
mse = calc_mse(prediction, ytest)

#  It makes sense to normalize by the variance of the
# targets of the test cases to obtain the standardized mean squared error (SMSE).
# This causes the trivial method of guessing the mean of the training targets to SMSE
# # have a SMSE of approximately 1.


smse = mse / v_y
print(f'MSE={mse:.2f}, SMSE={smse:.2f}')





MSE=22.68, SMSE=0.10


In [42]:
# We follow sec 2.5 of https://gaussianprocess.org/gpml/chapters/RW2.pdf
# The inputs were linearly rescaled to have zero mean and unit variance on the training set.
# The outputs were centered so as to have zero mean on the training set.
# SMSE is the MSE / var(ytrain)

def calc_mse(prediction, Y):
    mse = jnp.mean(jnp.square(prediction - Y))
    return mse

def add_col_ones(X):
    ones_column = jnp.ones((X.shape[0], 1))
    return jnp.hstack((ones_column, X))

scaler = preprocessing.StandardScaler().fit(data['X_train'])
Xtrain = add_col_ones(scaler.transform(data['X_train']))
Xtest = add_col_ones(scaler.transform(data['X_test']))
ytrain, ytest = data['Y_train'], data['Y_test']
mu_y, v_y = jnp.mean(ytrain), jnp.var(ytrain)
ytrain, ytest = ytrain - mu_y, ytest - mu_y

params, residuals, rank, s = np.linalg.lstsq(Xtrain, ytrain, rcond=None)
prediction = Xtest @ params 
mse = calc_mse(prediction, ytest)

model = sklearn.linear_model.LinearRegression()
model.fit(Xtrain, ytrain)
prediction = model.predict(Xtest)
mse_sklearn = calc_mse(prediction, ytest)

print(f'MSE(jax)={mse:.2f}, MSE(sklearn)={mse_sklearn:.2f}, SMSE={mse/v_y:.2f}')






MSE=22.68, SMSE=0.10


In [55]:
gauss_log_likelihood = lambda mean, cov, y: \
    jax.scipy.stats.norm.logpdf(y, mean, jnp.sqrt(jnp.diag(cov))).sum()

def nll_gauss(params, x, y):
    mu_y, v_t, w = params
    m = mu_y * jnp.eye(1)
    c = v_y * jnp.eye(1)
    return -gauss_log_likelihood(m, c, y)

def nll_linreg(params, x, y):
    mu_y, v_t, w = params
    m = jnp.dot(w, x) * jnp.eye(1)
    c = v_y * jnp.eye(1)
    return -gauss_log_likelihood(m, c, y)


def compute_regression_baselines(Xtrain, ytrain, Xtest, ytest):
    mu_y, v_y = jnp.mean(ytrain), jnp.var(ytrain)
    #  model = sklearn.linear_model.LinearRegression() 
    w, residuals, rank, s = np.linalg.lstsq(Xtrain, ytrain, rcond=None) # model.fit(Xtrain, ytrain)
    #prediction = Xtest @ w # prediction = model.predict(Xtest)
    params = (mu_y, v_y, w)

    nll_te_gauss = jnp.mean(jax.vmap(nll_baseline, (None, 0, 0))(params, Xtest, ytest))
    nll_te_linreg = jnp.mean(jax.vmap(nll_linreg, (None, 0, 0))(params, Xtest, ytest))
    return nll_te_gauss, nll_te_linreg



nll_te_gauss, nll_te_linreg = compute_regression_baselines(Xtrain, ytrain, Xtest, ytest)
msll = nll_te_linreg - nll_te_gauss
print(nll_te_linreg, msll)


3.6768196 -1.7491357


In [27]:

#https://github.com/JohDonald/SARCOS-problem/blob/main/SARCOS_Problem_ML.ipynb
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


sarcos = data['sarcos']
xt_, xtst_, yt, ytst = train_test_split(sarcos[:,:-1], sarcos[:,-1], train_size=0.9, random_state=101)

ntest = ytst.shape[0]
print(ntest)
ntest = 50 # following notebook
xtst_, ytst = xtst_[:ntest], ytst[:ntest]


scaler = preprocessing.StandardScaler().fit(xt_)
xt, xtst = scaler.transform(xt_), scaler.transform(xtst_)

def calc_mse(Prediction, Y):
    mse = np.mean(np.square(Prediction - Y))
    return mse



from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(xt,yt)
prediction = model.predict(xtst)
print(prediction.shape)
print(calc_mse(prediction, ytst))

4449
(50,)
16.766759879376583


In [8]:
import os
import argparse
import os
import itertools
import pandas as pd
from pathlib import Path
import os
import datetime

def list_subdirectories(directory):
    return [name for name in os.listdir(directory)
            if os.path.isdir(os.path.join(directory, name))]

src = '/teamspace/jobs'
dst = '/teamspace/studios/this_studio/jobs'
dirs = list_subdirectories(src)
for old_name in dirs:
    parts = old_name.split('-')
    num = parts[2]
    new_name = f'expt_timing2-{num}'
    dst_path = Path(dst, new_name, 'work')
    print(f'\n Creating {str(dst_path)}')
    dst_path.mkdir(parents=True, exist_ok=True)

    #cmd = f'mv {directory}/{old_name} {directory}/{new_name}'
    cmd = f'sudo cp -r {src}/{old_name}/work/*.* {dst}/{new_name}/work'
    print(cmd)
    os.system(cmd)


 Creating /teamspace/studios/this_studio/jobs/expt_timing2-13/work
sudo cp -r /teamspace/jobs/expt-timing2-13/work/*.* /teamspace/studios/this_studio/jobs/expt_timing2-13/work

 Creating /teamspace/studios/this_studio/jobs/expt_timing2-10/work
sudo cp -r /teamspace/jobs/expt-timing2-10/work/*.* /teamspace/studios/this_studio/jobs/expt_timing2-10/work

 Creating /teamspace/studios/this_studio/jobs/expt_timing2-02/work
sudo cp -r /teamspace/jobs/expt-timing2-02/work/*.* /teamspace/studios/this_studio/jobs/expt_timing2-02/work

 Creating /teamspace/studios/this_studio/jobs/expt_timing2-00/work
sudo cp -r /teamspace/jobs/expt-timing2-00/work/*.* /teamspace/studios/this_studio/jobs/expt_timing2-00/work

 Creating /teamspace/studios/this_studio/jobs/expt_timing2-15/work
sudo cp -r /teamspace/jobs/expt-timing2-15/work/*.* /teamspace/studios/this_studio/jobs/expt_timing2-15/work

 Creating /teamspace/studios/this_studio/jobs/expt_timing2-41/work
sudo cp -r /teamspace/jobs/expt-timing2-41/work